# EDA phase

## CSV file structure

The CSV files downloaded from the IMSS website follow the same structre for all months of each year. Therefore, for this exploration phase, a single file will be used to get a better understanding of how the data is stored in these.

In [1]:
import pandas as pd
import os
import json

In [79]:
# Get the IMSS files dtypes json and import it as a dictionary
with open(os.path.join(os.getcwd(),'IMSS_files_dtypes.json')) as dtypes_json:
    IMSS_files_dtypes = json.load(dtypes_json)

# Use getcwd to get the current folder location, then change the final folder in path to access the Scraping folder
IMSS_files_location = os.path.join(os.getcwd().replace('EDA','Scraping'),'IMSS_Files')

# Get list of files in target location
IMSS_files = os.listdir(IMSS_files_location)

# Get a file from the list
IMSS_first_file = os.path.join(IMSS_files_location, IMSS_files[0])
print(IMSS_first_file)

# Create dataframe from the first IMSS file
IMSS_df = pd.read_csv(IMSS_first_file, sep='|', encoding='latin-1',dtype=IMSS_files_dtypes)

C:\Users\J-D-S\Documents\Projects\IMSS-Salary-Analysis\Scraping\IMSS_Files\asg-2021-01-31.csv


# NaN value exploration
Below, the dataframe is printed to see its total entries and features. Some of these contain NaN values, which must be explored to determine the effect these have on the dataframe

In [80]:
# Print the dataframe
IMSS_df

,cve_delegacion,cve_subdelegacion,cve_entidad,cve_municipio,sector_economico_1,sector_economico_2,sector_economico_4,tamaño_patron,sexo,rango_edad,...,ta_sal,teu_sal,tec_sal,tpu_sal,tpc_sal,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc
0,1,1,1,A01,<NA>,<NA>,<NA>,NaN,1,E1,...,0,0,0,0,0,0.00,0.00,0.0,0.00,0.0
1,1,1,1,A01,<NA>,<NA>,<NA>,NaN,1,E10,...,0,0,0,0,0,0.00,0.00,0.0,0.00,0.0
2,1,1,1,A01,<NA>,<NA>,<NA>,NaN,1,E11,...,0,0,0,0,0,0.00,0.00,0.0,0.00,0.0
3,1,1,1,A01,<NA>,<NA>,<NA>,NaN,1,E12,...,0,0,0,0,0,0.00,0.00,0.0,0.00,0.0
4,1,1,1,A01,<NA>,<NA>,<NA>,NaN,1,E13,...,0,0,0,0,0,0.00,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4557040,40,58,9,NaN,9,99,9900,S3,2,E6,...,1,1,0,0,0,679.22,679.22,0.0,0.00,0.0
4557041,40,58,9,NaN,9,99,9900,S3,2,E7,...,1,0,0,1,0,1416.55,0.00,0.0,1416.55,0.0
4557042,40,58,9,NaN,9,99,9900,S3,2,E8,...,1,0,0,1,0,2172.00,0.00,0.0,2172.00,0.0
4557043,40,58,9,NaN,9,99,9900,S3,2,E9,...,1,0,0,1,0,2172.00,0.00,0.0,2172.00,0.0


In [37]:
# Get the number of empty entries for each of the dataframe's columns
for column in IMSS_df.columns:
    empty_entries_in_column = IMSS_df[column].isna().sum()
    if empty_entries_in_column > 0:
        print(f'{column}')
        print(f'Number of empty rows in the {column} colum: {empty_entries_in_column}')
        print(f'Percentage of missing information: {100 * empty_entries_in_column / len(IMSS_df)}%')
        print(f'----------------------------------------------------------------------------------')

cve_municipio
Number of empty rows in the cve_municipio colum: 508966
Percentage of missing information: 11.168772746374021%
----------------------------------------------------------------------------------
sector_economico_1
Number of empty rows in the sector_economico_1 colum: 16477
Percentage of missing information: 0.3615720274871106%
----------------------------------------------------------------------------------
sector_economico_2
Number of empty rows in the sector_economico_2 colum: 16477
Percentage of missing information: 0.3615720274871106%
----------------------------------------------------------------------------------
sector_economico_4
Number of empty rows in the sector_economico_4 colum: 16477
Percentage of missing information: 0.3615720274871106%
----------------------------------------------------------------------------------
tamaño_patron
Number of empty rows in the tamaño_patron colum: 26663
Percentage of missing information: 0.5850940686343892%
-----------------

## Empty entries
The previous for loop serves to count how many of the features in the dataset have empty values. These features are:
* cve_municipio
* sector_economico_1
* sector_economico_2
* sector_economico_4
* tamaño_patron
* rango_salarial
* rango_uma

cve_municipio is the code that's assigned for each city of its respective state. Therefore a separate dataframe will be created to analyze these missing values. It should be noticed that these values are only missing for the specific city but not the state. Depending on the required granularity of the information, these could be either removed if worked on a city-specific scope, or kept if the data will be used in a state-specific scale.

sector_economico_1,sector_economico_2 and sector_economico_4 have the same amount of missing values, therefore its likely that these missing values are all found in the same rows. These 3 features will be grouped in a different dataframe to find where these missing values are from.

tamaño_patron will be analyzed in its own dataframe since its missing value count is not related to the other features.

rango_salarial and rango_uma are different calculations for the salary. This is because the minimum wage, and UMA have a different value. These 2 features will be analyzed in a different dataframe

In [42]:
## Create function that returns a dataframe with the missing values for the specified feature
def nan_value_analysis(source_df, feature):    
    # Create a subset of the main dataframe where the target feature is NaN
    nan_feature_df = source_df.loc[source_df[feature].isna()]
    return nan_feature_df

In [43]:
nan_sector_economico_df = nan_value_analysis(IMSS_df,'sector_economico_1')

In [44]:
nan_sector_economico_df.describe(exclude='object')

,cve_delegacion,cve_subdelegacion,cve_entidad,sector_economico_1,sector_economico_2,sector_economico_4,sexo,asegurados,no_trabajadores,ta,...,ta_sal,teu_sal,tec_sal,tpu_sal,tpc_sal,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc
count,16477.000000,16477.000000,16477.000000,0,0,0,16477.000000,16477.000000,16477.000000,16477.0,...,16477.0,16477.0,16477.0,16477.0,16477.0,16477.0,16477.0,16477.0,16477.0,16477.0
mean,18.798507,13.450810,17.655216,<NA>,<NA>,<NA>,1.503429,482.520544,482.520544,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,9.061138,17.906699,8.132022,<NA>,<NA>,<NA>,0.500003,3068.243464,3068.243464,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,1.000000,1.000000,<NA>,<NA>,<NA>,1.000000,1.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,12.000000,1.000000,12.000000,<NA>,<NA>,<NA>,1.000000,3.000000,3.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,17.000000,6.000000,16.000000,<NA>,<NA>,<NA>,2.000000,26.000000,26.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,26.000000,17.000000,24.000000,<NA>,<NA>,<NA>,2.000000,170.000000,170.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,40.000000,80.000000,32.000000,<NA>,<NA>,<NA>,2.000000,103043.000000,103043.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### NaN Sector Economico Values
As seen in the descriptive summary of the dataframe, the rows with a missing sector_economico feature do not have relevant information to the analysis. This is noticeable by going through the ta and ta_sal columns. The minimum and maximum value for them is 0, which means that all rows in this subset of the dataframe have 0 as their value.

These columns can be safely dropped. Moreover, they make up less than 0.5% of the information, thus their removal will not cause any bias.

In [39]:
nan_cve_municipio_df = nan_value_analysis(IMSS_df,'cve_municipio')

In [41]:
nan_cve_municipio_df.describe(exclude='object')

,cve_delegacion,cve_subdelegacion,cve_entidad,sector_economico_1,sector_economico_2,sector_economico_4,sexo,asegurados,no_trabajadores,ta,...,ta_sal,teu_sal,tec_sal,tpu_sal,tpc_sal,masa_sal_ta,masa_sal_teu,masa_sal_tec,masa_sal_tpu,masa_sal_tpc
count,508966.000000,508966.000000,508966.0,508686.000000,508686.000000,508686.000000,508966.000000,508966.000000,508966.000000,508966.000000,...,508966.000000,508966.000000,508966.0,508966.000000,508966.000000,5.089660e+05,508966.000000,508966.0,5.089660e+05,508966.000000
mean,39.531478,30.825495,9.0,6.062321,63.966413,-2042.938496,1.431648,9.683268,3.360841,6.322426,...,6.311734,0.897376,0.0,5.414311,0.000047,3.385748e+03,399.578377,0.0,2.986161e+03,0.008841
std,0.499009,23.564121,0.0,2.037210,22.288018,2228.024804,0.495306,430.234775,429.460584,26.609510,...,26.593177,9.811795,0.0,22.959133,0.009084,1.946628e+04,4081.195102,0.0,1.821526e+04,1.719511
min,39.000000,1.000000,9.0,0.000000,1.000000,101.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
25%,39.000000,11.000000,9.0,4.000000,42.000000,4201.000000,1.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,0.0,1.000000,0.000000,4.664425e+02,0.000000,0.0,3.569800e+02,0.000000
50%,40.000000,16.000000,9.0,6.000000,67.000000,6707.000000,1.000000,2.000000,0.000000,2.000000,...,2.000000,0.000000,0.0,1.000000,0.000000,1.030570e+03,0.000000,0.0,8.741500e+02,0.000000
75%,40.000000,56.000000,9.0,8.000000,84.000000,8401.000000,2.000000,4.000000,0.000000,4.000000,...,4.000000,0.000000,0.0,3.000000,0.000000,2.172000e+03,0.000000,0.0,1.952245e+03,0.000000
max,40.000000,58.000000,9.0,9.000000,99.000000,9900.000000,2.000000,103043.000000,103043.000000,2526.000000,...,2526.000000,2162.000000,0.0,2523.000000,4.000000,2.748879e+06,697094.170000,0.0,2.575386e+06,623.980000


In [46]:
nan_cve_municipio_df.cve_delegacion.unique()

<IntegerArray>
[39, 40]
Length: 2, dtype: Int8

In [48]:
nan_cve_municipio_df.cve_entidad.unique()

<IntegerArray>
[9]
Length: 1, dtype: Int8

In [49]:
nan_cve_municipio_df.ta.sum()

3217900

In [50]:
nan_cve_municipio_df.rango_salarial.value_counts()

W2     160268
W3      71851
W4      54615
W5      42413
W6      28871
W7      26959
W8      20750
W16     16511
W9      16238
W10     15980
W11     11964
W12     11023
W13     10310
W15      9484
W14      8372
W1       2639
Name: rango_salarial, dtype: int64